In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os
import pandas as pd
import json
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
from datasets import Dataset, DatasetDict

In [2]:
from transformers import TrainingArguments, Trainer

2023-06-08 09:50:55.110974: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 09:50:55.161381: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 09:50:56.482095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
data_directory = "data/"
home_directory = "../"

NUM_EPOCHS = 10
BATCH_SIZE = 16

In [4]:
def get_articles(raw_data):
    data = []
    for each_row in raw_data['articles']:
        data.append(each_row['content'])
    return data

def get_split_data(split_data):
    data = []
    labels = []
    for i, news_src in enumerate(split_data['json_file_path']):
        raw_data = json.loads(open(os.path.join(home_directory, data_directory, news_src)).read())
        cur_data = get_articles(raw_data)
        data.extend(cur_data)
        labels.extend([split_data['label'][i] for _ in range(len(cur_data))])
    final_data = [data, labels]
    return final_data

### Factuality

In [5]:
fact_train_file_path = "data/task_4/task_4_news_media_factuality_train.tsv"
fact_dev_file_path = "data/task_4/task_4_news_media_factuality_dev.tsv"
fact_test_gold_file_path = "task_4_news_media_factuality_test.tsv"

In [6]:
fact_train_raw_data = pd.read_csv(os.path.join(home_directory, data_directory, fact_train_file_path), sep = "\t")
fact_dev_raw_data = pd.read_csv(os.path.join(home_directory, data_directory, fact_dev_file_path), sep = "\t")
fact_test_raw_data = pd.read_csv(os.path.join(home_directory, data_directory, fact_test_gold_file_path), sep = "\t")

In [7]:
fact_train_raw_data.shape

(947, 3)

In [8]:
fact_train_data = get_split_data(fact_train_raw_data)
fact_dev_data  = get_split_data(fact_dev_raw_data)
fact_test_data = get_split_data(fact_test_raw_data)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels = 3)

Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a

In [10]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [11]:
model.to(device)

MPNetForSequenceClassification(
  (mpnet): MPNetModel(
    (embeddings): MPNetEmbeddings(
      (word_embeddings): Embedding(30527, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MPNetEncoder(
      (layer): ModuleList(
        (0): MPNetLayer(
          (attention): MPNetAttention(
            (attn): MPNetSelfAttention(
              (q): Linear(in_features=768, out_features=768, bias=True)
              (k): Linear(in_features=768, out_features=768, bias=True)
              (v): Linear(in_features=768, out_features=768, bias=True)
              (o): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [12]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, return_tensors="pt")

In [13]:
df_train = pd.DataFrame({
    'text': fact_train_data[0],
    'label': fact_train_data[1]
})

df_dev = pd.DataFrame({
    'text': fact_dev_data[0],
    'label': fact_dev_data[1]
})

df_test = pd.DataFrame({
    'text': fact_test_data[0],
    'label': fact_test_data[1]
})

In [14]:
train_ds = Dataset.from_pandas(df_train)
dev_ds = Dataset.from_pandas(df_dev)
test_ds = Dataset.from_pandas(df_test)

In [15]:
tokenized_train_ds = train_ds.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dev_ds = dev_ds.map(tokenize, batched=True, remove_columns=["text"])
tokenized_test_ds = test_ds.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/7948 [00:00<?, ? examples/s]

Map:   0%|          | 0/1049 [00:00<?, ? examples/s]

Map:   0%|          | 0/1054 [00:00<?, ? examples/s]

In [16]:
import evaluate

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.argmax(torch.from_numpy(predictions), dim = -1)

    return {'mse_error': nn.L1Loss()(predictions.to(torch.float64), torch.FloatTensor(labels))}

In [17]:
import os

os.environ["WANDB_PROJECT"] = "ugrip-nlp1"

In [18]:
training_args = TrainingArguments(
    output_dir='sbert-factuality',
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='sbert-factuality-logs',
    load_best_model_at_end=True,
    save_total_limit=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to = "none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_dev_ds,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

/home/neemesh.yadav/.conda/envs/ugrip/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse Error
1,No log,0.793026,0.399428
2,0.877900,0.943838,0.427073
3,0.717500,0.906533,0.388942
4,0.538000,1.033597,0.395615
5,0.382000,1.272376,0.386082
6,0.247200,1.689255,0.399428
7,0.159700,1.583540,0.376549
8,0.112200,1.827421,0.370829
9,0.070800,1.995791,0.378456
10,0.040600,2.046316,0.377502


TrainOutput(global_step=4970, training_loss=0.3190496197168976, metrics={'train_runtime': 3492.2179, 'train_samples_per_second': 22.759, 'train_steps_per_second': 1.423, 'total_flos': 2.091225444102144e+16, 'train_loss': 0.3190496197168976, 'epoch': 10.0})

In [28]:
preds = trainer.predict(tokenized_test_ds)

In [29]:
preds = preds.predictions

In [31]:
pred_labs = torch.argmax(torch.from_numpy(preds), dim = -1)

In [32]:
pred_labs = pred_labs.to(torch.float64)

In [33]:
print("Accuracy: ", sum(torch.FloatTensor(tokenized_test_ds['label']) == pred_labs)/pred_labs.shape[0])

Accuracy:  tensor(0.6632)


In [34]:
print("MAE: ", nn.L1Loss()(pred_labs, torch.FloatTensor(tokenized_test_ds['label'])).item())

MAE:  0.42125237191650855


### Political Bias

In [26]:
bias_A_train_file_path = "data/task_3A/task_3A_news_article_bias_train.tsv"
bias_A_dev_file_path = "data/task_3A/task_3A_news_news_article_bias_dev.tsv"
bias_A_test_gold_file_path = "task_3A_news_news_article_bias_test.tsv"

In [27]:
# bias_A_train_raw_data = pd.read_csv(os.path.join(home_directory, data_directory, bias_A_train_file_path), sep = "\t")
# bias_A_dev_raw_data = pd.read_csv(os.path.join(home_directory, data_directory, bias_A_dev_file_path), sep = "\t")
# bias_A_test_raw_data = pd.read_csv(os.path.join(home_directory, data_directory, bias_A_test_gold_file_path), sep = "\t")